In [33]:
#path

path = "valmiki-ramayana-only-hindi.pdf"

In [ ]:
#Searching for largest font size from middle of pdf to next 10% pages 

In [35]:
import time
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams, LTChar, LTTextBox
from pdfminer.converter import PDFPageAggregator

start_time = time.time() 

def is_digit(char):
    return char.isdigit()

def extract_font_info(layout):
    max_font_size = 0  # Variable to store the largest font size

    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox):
            for text_line in lt_obj:
                for character in text_line:
                    if isinstance(character, LTChar):
                        current_char = character.get_text()

                        # Skip digit characters
                        if is_digit(current_char):
                            continue

                        current_font_size = character.size

                        # Update max_font_size if the current font size is larger
                        if current_font_size > max_font_size:
                            max_font_size = current_font_size

    return max_font_size


# Open a PDF file.
with open(path, 'rb') as fp:
    parser = PDFParser(fp)
    document = PDFDocument(parser)

    # Check if the document allows text extraction. If not, abort.
    if not document.is_extractable:
        raise PDFTextExtractionNotAllowed

    # Create a PDF resource manager object that stores shared resources.
    rsrcmgr = PDFResourceManager()

    # Create a PDF device object.
    device = PDFDevice(rsrcmgr)

    # BEGIN LAYOUT ANALYSIS
    # Set parameters for analysis.
    laparams = LAParams()

    # Create a PDF page aggregator object.
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)

    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    total_pages = len(list(PDFPage.create_pages(document)))
    start_page = int(total_pages * 0.5)
    end_page = int(total_pages * 0.6)

    largest_font_size = 0  # Variable to store the largest font size across all pages

    for page_number, page in enumerate(PDFPage.create_pages(document)):
        if start_page <= page_number < end_page:
            interpreter.process_page(page)
            layout = device.get_result()
            current_page_max_font_size = extract_font_info(layout)

            # Update largest_font_size if the current page's max font size is larger
            if current_page_max_font_size > largest_font_size:
                largest_font_size = current_page_max_font_size

    print(f"Largest Font Size (excluding digits): {largest_font_size}")

print("--- %s seconds ---" % (time.time() - start_time))


Largest Font Size (excluding digits): 24.997499999999945
--- 40.71692252159119 seconds ---


In [ ]:
#extracting data into json from entire pdf

In [36]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLineHorizontal, LTAnno
import re
import json

start_time = time.time() 


temp_pdf = []
page_data = ""
temp_dict = {}

for page_layout in extract_pages(path):
    global temp_dict
    global page_data
    for element in page_layout:      # extracting the pages
        if isinstance(element, LTTextContainer):
            for text_line in element: # extracting the lines in pages
                if isinstance(text_line, LTTextLineHorizontal):
                    for chr in text_line:
                        if isinstance(chr, LTChar):
                            # Check for bold, italic, and hyperlink
                            if (chr.size >= largest_font_size) and ('Bold' in chr.fontname or 'Italic' in chr.fontname or text_line.get_text().isupper()):

                                # print(f"Outer Title: {text_line.get_text()}")
                                # exit()
                                if len(page_data) > 0:
                                    # print(f"Title: {text_line.get_text()}")
                                    temp_dict['data'] = page_data
                                    temp_pdf.append(temp_dict)
                                    temp_dict = {}
                                temp_dict["title"] = text_line.get_text()
                                # print("content======================")
                                # print(page_data)
                                
                                page_data = ""
                                has_content_started = True
                                break
                            elif (chr.size<largest_font_size):
                                page_data += text_line.get_text()
                                # print("content======================")
                                # print(page_data)
                                break
# print(json.dumps(temp_pdf))
out_file = open("sw.json", "w") 
  
json.dump(temp_pdf, out_file, indent = 6) 
  
out_file.close()
print("--- %s seconds ---" % (time.time() - start_time))


--- 400.58130288124084 seconds ---


In [ ]:
#extracting data into json from entire pdf ## add encoding for hindi (but not working 🥲)

In [37]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLineHorizontal, LTAnno
import re
import json

start_time = time.time() 


temp_pdf = []
page_data = ""
temp_dict = {}

for page_layout in extract_pages(path):
    global temp_dict
    global page_data
    for element in page_layout:      # extracting the pages
        if isinstance(element, LTTextContainer):
            for text_line in element: # extracting the lines in pages
                if isinstance(text_line, LTTextLineHorizontal):
                    for chr in text_line:
                        if isinstance(chr, LTChar):
                            # Check for bold, italic, and hyperlink
                            if (chr.size >= largest_font_size) and ('Bold' in chr.fontname or 'Italic' in chr.fontname or text_line.get_text().isupper()):

                                # print(f"Outer Title: {text_line.get_text()}")
                                # exit()
                                if len(page_data) > 0:
                                    # print(f"Title: {text_line.get_text()}")
                                    temp_dict['data'] = page_data
                                    temp_pdf.append(temp_dict)
                                    temp_dict = {}
                                temp_dict["title"] = text_line.get_text()
                                # print("content======================")
                                # print(page_data)
                                
                                page_data = ""
                                has_content_started = True
                                break
                            elif (chr.size<largest_font_size):
                                page_data += text_line.get_text()
                                # print("content======================")
                                # print(page_data)
                                break

out_file = open("swen.json", "w", encoding='utf-8')  # Add encoding='utf-8'
json.dump(temp_pdf, out_file, indent=6, ensure_ascii=False)  # Add ensure_ascii=False
out_file.close()

In [ ]:
## for custom starting and ending pages 

In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLineHorizontal, LTAnno
import re
import json

start_time = time.time() 


temp_pdf = []
page_data = ""
temp_dict = {}

start_page_number = 9
end_page_number = 15
# for page_number, page_layout in enumerate(extract_pages(path), start=1):
#     if page_number == target_page_number:
#         for element in page_layout:


# for page_layout in extract_pages(path):
for page_number, page_layout in enumerate(extract_pages(path), start=1):

    global temp_dict
    global page_data
#     if page_number == target_page_number:
    if start_page_number <= page_number <= end_page_number:
        for element in page_layout:      # extracting the pages
            if isinstance(element, LTTextContainer):
                for text_line in element: # extracting the lines in pages
                    if isinstance(text_line, LTTextLineHorizontal):
                        for chr in text_line:
                            if isinstance(chr, LTChar):
                                # Check for bold, italic, and hyperlink
                                if (chr.size >= largest_font_size) and ('Bold' in chr.fontname or 'Italic' in chr.fontname or text_line.get_text().isupper()):

                                    # print(f"Outer Title: {text_line.get_text()}")
                                    # exit()
                                    if len(page_data) > 0:
                                        # print(f"Title: {text_line.get_text()}")
                                        temp_dict['data'] = page_data
                                        temp_pdf.append(temp_dict)
                                        temp_dict = {}
                                    temp_dict["title"] = text_line.get_text()
                                    # print("content======================")
                                    # print(page_data)

                                    page_data = ""
                                    has_content_started = True
                                    break
                                elif (chr.size<largest_font_size):
                                    page_data += text_line.get_text()
                                    # print("content======================")
                                    # print(page_data)
                                    break

out_file = open("pART13.json", "w") 
  
json.dump(temp_pdf, out_file, indent = 6) 
  
out_file.close()

In [ ]:
# checking font size manually from custom page number

In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLineHorizontal
import time

start_time = time.time()
# path = 'your_pdf_file.pdf'  # Replace with the actual path to your PDF file

target_page_number = 10

for page_number, page_layout in enumerate(extract_pages(path), start=1):
    if page_number == target_page_number:
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if isinstance(text_line, LTTextLineHorizontal):
                        for chr in text_line:
                            if isinstance(chr, LTChar):
                                print(f"Font Size: {chr.size}, Text: {text_line.get_text()}")
                                # Add more debugging information as needed

                                # Your existing logic here

print("--- %s seconds ---" % (time.time() - start_time))
